# LFP example 


In [1]:
import altair as alt
from bayes_window.generative_models import *
from bayes_window.visualization import plot_data, plot_data_slope_trials
from bayes_window import BayesWindow, models

## Make and visualize model oscillation power
40 trials of "theta power" is generated for every animal. It is drawn randomly as a poisson process. 

This is repeated for "stimulation" trials, but poisson rate is higher.

In [2]:
# Draw some fake data:
df, df_monster, index_cols, _ = generate_fake_lfp(mouse_response_slope=9, n_trials=30)

Mice vary in their baseline power. 

Higher-baseline mice tend to have smaller stim response:

In [3]:
c1=plot_data(df=df,x='stim',y='Log power').properties(width=60)
c2=plot_data_slope_trials(df=df,x='stim',y='Log power',color=None,detail='i_trial')
(c1+c2).facet(column='mouse')

alt.FacetChart(...)

In [4]:
plot_data(df=df,x='stim',y='Log power',color='mouse').properties(width=80)

alt.LayerChart(...)

## Fit a Bayesian hierarchical model and plot slopes
In a hierarchical model, parameters are viewed as a sample from a population distribution of parameters. Thus, we view them as being neither entirely different or exactly the same. This is ***partial pooling***:

![hierarchical](../motivation/parpooled.png)
This model allows intercepts to vary across mouse, according to a random effect. We just add a fixed slope for the predictor (i.e all mice will have the same slope):

$$y_i = \alpha_{j[i]} + \beta x_{i} + \epsilon_i$$

where $j$ is mouse index and

$$\epsilon_i \sim N(0, \sigma_y^2)$$

and the intercept random effect:

$$\alpha_{j[i]} \sim N(\mu_{\alpha}, \sigma_{\alpha}^2)$$

We set a separate intercept for each mouse, but rather than fitting separate regression models for each mouse, multilevel modeling **shares strength** among mice, allowing for more reasonable inference in mice with little data.

The wrappers in this library allow us to fit and plot this inference in just three lines of code. Under the hood, it uses the following Numpyro code:
```python
# Normal intercepts 
a = numpyro.sample('a', dist.Normal(0, 1))
a_subject = numpyro.sample('a_subject', dist.Normal(jnp.tile(0, n_subjects), 1))

# Variances
sigma_a_subject = numpyro.sample('sigma_a', dist.HalfNormal(1))
sigma_obs = numpyro.sample('sigma_obs', dist.HalfNormal(1))

# Normal treatment
b = numpyro.sample('b_stim', dist.Normal(0, 1))

# Regression equation
slope = a + a_subject[group] * sigma_a + b * treatment 

# Sample variable
numpyro.sample('y', dist.Normal(theta, sigma_obs), obs=y)
```


Above is the contents of `model_hier_stim_one_codition.py`, the function passed as argument in line 4 below.

In [5]:
# Initialize:
window=BayesWindow(df, y='Log power', treatment='stim', group='mouse')
# Fit: 
window.fit_slopes(add_data=True, model=models.model_hier_stim_one_codition,
                  do_make_change='subtract', dist_y='student');

# Plot:
chart_power_difference = window.plot(independent_axes=False)
chart_power_difference

alt.LayerChart(...)

In this chart:

- The blue dot is the mean of posterior

- The black line is the 94% highest density interval

- The boxplot is made from difference between groups in the data (no fitting)


## Compare to non-bayesian approaches

ANOVA does not pick up the effect of stim as significant:

In [6]:
window.fit_anova();

Log_power~stim
             sum_sq    df         F    PR(>F)
stim      0.012908   1.0  0.506072  0.488529
Residual  0.357101  14.0       NaN       NaN


A linear mixed-effect model shows the effect of stim (slope) as significant. It includes random intercepts of mouse:

In [7]:
# Initialize:
window=BayesWindow(df, y='Log power', treatment='stim', group='mouse')
window.fit_lme(add_data=False)

Using formula Log_power ~ (1 | mouse) + (stim | mouse)
               Coef. Std.Err.       z  P>|z|  [0.025 0.975]
Intercept      1.970    0.041  48.303  0.000   1.890  2.050
1 | mouse     -0.019    0.037  -0.512  0.608  -0.091  0.053
stim | mouse   0.080    0.036   2.234  0.025   0.010  0.150
Group Var      0.002    0.006                              


In [8]:
window.plot_posteriors_slopes()

alt.LayerChart(...)

## Inspect Bayesian result further
We named intercept _a\_subject_, so now we can plot it for each animal

In [9]:
# Initialize:
window=BayesWindow(df, y='Log power', treatment='stim', group='mouse')
# Fit slopes again: 
window.fit_slopes(add_data=True, model=models.model_hier_stim_one_codition,
                  do_make_change='subtract', dist_y='normal');
intercepts=window.trace.posterior['a_subject'].mean(['chain','draw']).to_dataframe().reset_index()
# Plot
chart_intercepts=alt.Chart(intercepts).mark_bar().encode(
    x=alt.X('mouse:O',title='Mouse'),
    y=alt.Y('a_subject', title='Intercept')
)
chart_intercepts

alt.Chart(...)

Our plotting backend's flexibility allows us to easily concatenate multiple charts in the same figures with the | operator:

In [10]:
chart_intercepts | chart_power_difference

alt.HConcatChart(...)

## Include all samples in each trial
The mean of every one of the 30 trials we drew for each mouse is a manifestation of the same underlying process that generates power for each mouse. Let's try to include all samples that come in each trial

In [11]:
# Initialize:
window=BayesWindow(df_monster,#.groupby(['mouse','stim', ]).sample(n=230),
                   y='Log power', treatment='stim', group='mouse')
# Fit: 
window.fit_slopes(add_data=True, model=models.model_hier_stim_one_codition,num_warmup=1000,
                  do_make_change='subtract', dist_y='normal', progress_bar=True);

# Plot:
window.plot(independent_axes=False)


sample: 100%|██████████| 2000/2000 [05:08<00:00,  6.49it/s, 223 steps of size 9.86e-03. acc. prob=0.95] 


alt.LayerChart(...)

Much tighter credible intervals here! 

Same with linear mixed model: 

In [14]:
alt.data_transformers.disable_max_rows()
window=BayesWindow(df_monster,
                   y='Log power', treatment='stim', group='mouse')
window.fit_lme()


window.plot_posteriors_slopes()

Using formula Log_power ~ (1 | mouse) + (stim | mouse)
              Coef. Std.Err.       z  P>|z|  [0.025 0.975]
Intercept     1.972    0.043  45.598  0.000   1.887  2.057
1 | mouse     0.004    0.010   0.372  0.710  -0.016  0.023
stim | mouse  0.057    0.003  17.954  0.000   0.051  0.064
Group Var     0.004    0.007                              


alt.LayerChart(...)